In [5]:
import os

mind_data_path = "/kaggle/input/mind-data/mind_data"  
os.makedirs(mind_data_path, exist_ok=True) 

In [6]:
!pip install tensorflow==2.15.0 "numpy<2.0"

  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: k

In [7]:
# Cài cuDNN 8.9 từ pip
!pip uninstall tensorflow -y
!pip install nvidia-cudnn-cu12==8.9.7.29
!pip install tensorflow==2.15.0

# Thiết lập LD_LIBRARY_PATH
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

print("✅ cuDNN 8.9 installed. Please RESTART KERNEL!")

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 8.9.7.29 which is incompatible.
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.3 MB)
ERROR: pip's d

In [8]:
import os
cudnn_path = '/usr/local/lib/python3.11/dist-packages/nvidia/cudnn/lib'
os.environ['LD_LIBRARY_PATH'] = f"{cudnn_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))

2026-01-09 03:31:42.498223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-09 03:31:42.498290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-09 03:31:42.499742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# Cell 1: Monkey patch để force embedding lên GPU
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# Monkey patch Embedding layer để force GPU placement
original_embedding_init = tf.keras.layers.Embedding.__init__

def patched_embedding_init(self, *args, **kwargs):
    # Force embeddings on GPU
    with tf.device('/GPU:0'):
        original_embedding_init(self, *args, **kwargs)

tf.keras.layers.Embedding.__init__ = patched_embedding_init

print("✅ Patched Embedding layer to use GPU")

✅ Patched Embedding layer to use GPU


In [10]:
# Cell 2: Setup  
from recommenders.models.newsrec.newsrec_utils import prepare_hparams    
from recommenders.models.newsrec.models.naml import NAMLModel  # Thay đổi  
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator  # Thay đổi  
  
train_news_file = os.path.join(mind_data_path, "train", "news.tsv")    
train_behaviors_file = os.path.join(mind_data_path, "train", "behaviors.tsv")    
valid_news_file = os.path.join(mind_data_path, "valid", "news.tsv")    
valid_behaviors_file = os.path.join(mind_data_path, "valid", "behaviors.tsv")    
wordEmb_file = os.path.join(mind_data_path, "utils", "embedding_all.npy")  # Thay đổi  
userDict_file = os.path.join(mind_data_path, "utils", "uid2index.pkl")    
wordDict_file = os.path.join(mind_data_path, "utils", "word_dict_all.pkl")  # Thay đổi  
vertDict_file = os.path.join(mind_data_path, "utils", "vert_dict.pkl")  # Thêm  
subvertDict_file = os.path.join(mind_data_path, "utils", "subvert_dict.pkl")  # Thêm  
yaml_file = os.path.join(mind_data_path, "utils", "naml.yaml")  # Thay đổi  
  
hparams = prepare_hparams(    
    yaml_file,    
    wordEmb_file=wordEmb_file,    
    wordDict_file=wordDict_file,    
    userDict_file=userDict_file,  
    vertDict_file=vertDict_file,  # Thêm  
    subvertDict_file=subvertDict_file,  # Thêm  
    epochs=1,  
    batch_size=384  
)    
  
# Create model with GPU  
with tf.device('/GPU:0'):  
    iterator = MINDAllIterator  # Thay đổi  
    model = NAMLModel(hparams, iterator, seed=42)  # Thay đổi  
  
print("✅ NAML Model created on GPU")

✅ NAML Model created on GPU


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# Cell 3: Load checkpoint
# checkpoint_path = "/kaggle/input/model-epoch-10-13/model/nrms_ckpt"
# model.model.load_weights(checkpoint_path)
# print(f"✅ Checkpoint loaded")

In [11]:
# Cell 3: Load checkpoint
checkpoint_path = "/kaggle/input/model-naml/model/nrms_ckpt"
model.model.load_weights(checkpoint_path)
print(f"✅ Checkpoint loaded")

✅ Checkpoint loaded


In [ ]:
# Cell 4: Training
print("🔥 Starting training...")
model.fit(  
    train_news_file,
    train_behaviors_file,
    valid_news_file,
    valid_behaviors_file
)
print("✅ Training completed!")

In [ ]:
# Đánh giá trên validation set  
eval_results = model.run_eval(valid_news_file, valid_behaviors_file)  
print("Kết quả đánh giá:")  
for metric, value in eval_results.items():  
    print(f"{metric}: {value:.4f}")

In [ ]:
model_path = os.path.join("/kaggle/working/", "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

In [12]:
from tqdm import tqdm
import numpy as np

test_behaviors_file = "/kaggle/input/mind-data/mind_data/test/behaviors.tsv"  
test_news_file = "/kaggle/input/mind-data/mind_data/test/news.tsv"

print("🔧 Patching iterator for test set (no labels)...")

# Backup original method
original_init = model.test_iterator.init_behaviors

def init_behaviors_no_labels(behaviors_file):
    """Modified init_behaviors for test set without labels"""
    model.test_iterator.histories = []
    model.test_iterator.imprs = []
    model.test_iterator.labels = []
    model.test_iterator.impr_indexes = []
    model.test_iterator.uindexes = []

    with open(behaviors_file, "r", encoding="utf-8") as rd:
        impr_index = 0
        for line in rd:
            uid, time, history, impr = line.strip("\n").split("\t")[-4:]

            # Parse history
            history = [model.test_iterator.nid2index[i] for i in history.split() if i in model.test_iterator.nid2index]
            history = [0] * (model.test_iterator.his_size - len(history)) + history[:model.test_iterator.his_size]

            # Parse impressions - TEST SET KHÔNG CÓ LABEL
            impr_news = []
            for item in impr.split():
                # Test set: chỉ có news_id, KHÔNG có "-0" hay "-1"
                if "-" in item:
                    # Validation/train set format: N12345-1
                    news_id = item.split("-")[0]
                else:
                    # Test set format: N12345
                    news_id = item
                
                if news_id in model.test_iterator.nid2index:
                    impr_news.append(model.test_iterator.nid2index[news_id])
            
            # Tạo dummy labels (không dùng cho test)
            label = [0] * len(impr_news)
            
            uindex = model.test_iterator.uid2index[uid] if uid in model.test_iterator.uid2index else 0

            model.test_iterator.histories.append(history)
            model.test_iterator.imprs.append(impr_news)
            model.test_iterator.labels.append(label)
            model.test_iterator.impr_indexes.append(impr_index)
            model.test_iterator.uindexes.append(uindex)
            impr_index += 1

# Apply patch
model.test_iterator.init_behaviors = init_behaviors_no_labels

print("✅ Iterator patched for test set")

# Run evaluationa
print("\n🔍 Running evaluation on test set...")
print("   Model will use trained weights to generate rankings")

group_impr_indexes, group_labels, group_preds = model.run_fast_eval(
    test_news_file, test_behaviors_file
)

print(f"\n✅ Generated predictions for {len(group_impr_indexes)} impressions")

# Write predictions
print("\n💾 Writing predictions to file...")
prediction_file = "/kaggle/working/prediction.txt"

with open(prediction_file, 'w') as f:  
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds), 
                                   total=len(group_impr_indexes),
                                   desc="Writing"):  
        # MIND competition format: impression_id bắt đầu từ 1
        impr_id = impr_index + 1
        
        # Calculate rankings (score cao nhất = rank 1)
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()  
        pred_rank_str = '[' + ','.join([str(i) for i in pred_rank]) + ']'  
        
        f.write(f"{impr_id} {pred_rank_str}\n")

print(f"\n✅ Prediction file saved to: {prediction_file}")

# Validate output
print("\n🔍 Validating output format...")
with open(prediction_file, 'r') as f:
    lines = f.readlines()
    print(f"   Total predictions: {len(lines)}")
    print(f"   First 3 lines:")
    for i in range(min(3, len(lines))):
        parts = lines[i].strip().split()
        print(f"      ImprID={parts[0]}, Rankings={parts[1][:50]}{'...' if len(parts[1]) > 50 else ''}")

print("\n✅ File ready to submit to MIND competition!")
print(f"   Download: {prediction_file}")

🔧 Patching iterator for test set (no labels)...
✅ Iterator patched for test set

🔍 Running evaluation on test set...
   Model will use trained weights to generate rankings


0it [00:00, ?it/s]/usr/local/lib/python3.11/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
120579it [03:45, 535.73it/s]
2370344it [1:22:00, 481.73it/s]
2370727it [02:34, 15326.04it/s]



✅ Generated predictions for 2370727 impressions

💾 Writing predictions to file...


Writing: 100%|██████████| 2370727/2370727 [00:29<00:00, 80174.95it/s]



✅ Prediction file saved to: /kaggle/working/prediction.txt

🔍 Validating output format...
   Total predictions: 2370727
   First 3 lines:
      ImprID=1, Rankings=[7,8,14,4,6,11,16,15,3,1,12,10,2,5,13,9]
      ImprID=2, Rankings=[6,3,5,2,4,7,1]
      ImprID=3, Rankings=[72,71,1,11,8,81,52,45,6,62,17,21,56,7,22,46,79,73...

✅ File ready to submit to MIND competition!
   Download: /kaggle/working/prediction.txt


In [ ]:
import zipfile
import os

# Đường dẫn file input (đã tạo ở bước trước) và output
source_file = "/kaggle/working/prediction.txt"
zip_output_path = "/kaggle/working/prediction.zip"


try:
    # Tạo file zip với chế độ nén ZIP_DEFLATED
    with zipfile.ZipFile(zip_output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # arcname='prediction.txt' đảm bảo file trong zip chỉ có tên là prediction.txt
        # chứ không chứa đường dẫn thư mục mẹ (/kaggle/working/...)
        zipf.write(source_file, arcname='prediction.txt')


    # (Tùy chọn) Kiểm tra nội dung bên trong file zip vừa tạo
    with zipfile.ZipFile(zip_output_path, 'r') as zipf:
        for info in zipf.infolist():
            print(f"File: {info.filename} | Size: {info.file_size / 1024:.2f} KB")

except Exception as e:
    print(e)